In [1]:
%gui asyncio

In [2]:
import asyncio    
from ipylgbst import LegoBoostWidget,LedColor,Port, Sensor
from ipywidgets import Output
from IPython.display import display

colors = [
    LedColor.pink,
    LedColor.purple,
    LedColor.blue,
    LedColor.lightblue,
    LedColor.cyan,
    LedColor.green,
    LedColor.yellow,
    LedColor.orange,
    LedColor.red,
]

In [3]:
boost = LegoBoostWidget()
boost.connect()
boost

Output()

LegoBoostWidget()

# Change Color of LED

In [4]:

out = Output()

async def main(lane, log):

    for color in colors:
        log(f"set color to {color.value}")
        await lane.set_led(color)
        await asyncio.sleep(0.5)
        
boost.run(main);

Output()

# Drive a polygon like path

In [5]:
speed = 26
d_forward  = 200
d_rotate = 300
async def main(lane,log):  
    for i in range(4):
        await lane.set_led("red")
        await lane.motor_angle_multi(angle=d_forward, power_a=speed,power_b=speed)
        await lane.set_led("green")
        await lane.motor_angle_multi(angle=d_rotate, power_a=speed,power_b=-1.0*speed)
        

f = boost.run(main)

Output()

# Buttons

In [5]:
import ipywidgets as widgets

button_up = widgets.Button(description='Up')
button_down = widgets.Button(description='Down')
button_left = widgets.Button(description='Left')
button_right = widgets.Button(description='Right')
button_stop  = widgets.Button(description='Stop')

box_ud = widgets.VBox([button_up,button_stop,button_down])
box = widgets.HBox([button_left,box_ud,button_right])


async def on_up(lane, out):
    await lane.motor_time_multi(10, 50, 50, wait=False)
    
async def on_down(lane, out):
    await lane.motor_time_multi(10, -50, -50, wait=False)
    
async def on_left(lane, out):
    await lane.motor_time_multi(10, -50, 50, wait=False)
    
async def on_right(lane, out):
    await lane.motor_time_multi(10, 50, -50, wait=False)
    
async def on_stop(lane, out):
    await lane.motor_time_multi(0, 0, 0, wait=False)
    
button_up.on_click(lambda _: boost.run(on_up))
button_down.on_click(lambda _: boost.run(on_down))
button_left.on_click(lambda _: boost.run(on_left))
button_right.on_click(lambda _: boost.run(on_right))
button_stop.on_click(lambda _: boost.run(on_stop))

box

# Sensors

In [9]:
has_bqplot = True
try:
    import bqplot
except ImportError:
    has_bqplot = False

if has_bqplot:
    from bqplot import pyplot as plt
    from bqplot import topo_load
    import numpy

    duration = 10.0
    dt = 0.10
    clip_value = 200

    sensor_values = []
    time_points = []

    plt.figure()
    plt.plot([], [])
    plt.xlim(0,duration)
    plt.ylim(0, clip_value)
    plt.xlabel("time")
    plt.ylabel("distance sensor")
    plt.show()

    async def poll_distance_sensor(lane, log):
        global sensor_values
        global time_points
        t = 0.0
        while t < duration:
            time_points.append(t)
            d = numpy.nan_to_num(await lane.get_distance(), clip_value)
            d = numpy.clip(d,0, clip_value)
            sensor_values.append(d)
            await asyncio.sleep(dt)
            t += dt
            plt.plot(time_points, sensor_values)

    boost.run(poll_distance_sensor);

Output()

# Concurrent Programs

In [8]:

async def set_leds(lane, log):

    for color in colors:
        log(f"set color to {color.value}")
        await lane.set_led(color)
        await asyncio.sleep(0.5)
        

speed = 26
d_forward  = 200
d_rotate = 300
async def drive(lane,log):  
    for i in range(4):
        await lane.motor_angle_multi(angle=d_forward, power_a=speed,power_b=speed)
        await lane.motor_angle_multi(angle=d_rotate, power_a=speed,power_b=-1.0*speed)


boost.run_concurrent([set_leds,drive]);

Output()